In [10]:
# !pip install yfinance # run this cell once

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import os 

# Estrategia: Combo Alfa

### Paso a paso del procedimiento de regresión

1) Se comienza obteniendo los retornos de la serie de tiempo de las acciones (o alfas) 
$$R_{is}; i = 1,...,N; s = 1,...,M+1.$$

2) Calcular los retornos netos de la media serial
$$Xis = R_{is} - \frac{1}{M+1} \sum^{M+1}_{s=1} Ris.$$

3) Calcular varianzas muestrales de los retornos
$$\sigma_{i}^{2}= C_{ii} = \frac{1}{M}\sum^{M+1}_{s=1} X^{2}_{is}.$$

4) Calcular los retornos netos de la media normalizados
$$Y_{is} = X_{is}/\sigma_{i}.$$

5) Mantener solo las primeras $M$ columnas en
$$Y_{is}: s = 1,...,M.$$

6) Ajustar por la media de corte transversal
$$Y_{is}: \Lambda_{is} = Y_{is} - \frac{1}{N}\sum^{N}_{j=1} Y_{js}.$$

7) Keep oMantener solo las primeras $M − 1$ columnas en 
$$\Lambda_{is}: s = 1,...,M - 1.$$

8) Tomar los retornos esperados de los alfas $E_{i}$ y normalizarlos 
$$\tilde{E}_{i} = Ei/\sigma_{i}.$$

9) Clacular los residuos $\tilde{\epsilon}_i$ de la regresión de $\tilde{E}_{i}$ sobre $\Lambda_{is}$.

10) Establecer las ponderaciones del portafolio como $\omega_{i} =  \eta \tilde{\epsilon}_{i}/\sigma_{i}.$

11) Establecer el coeficiente de normalización $\eta$ tal que
$$\sum^{N}_{i=1} |\omega_{i}| = 1.$$

In [2]:
def weights(returns, rm_overall=True, d=50):
    """
    
    returns: 
    """
    ret_matrix = returns.values
    
    #2. Compute the serially demeaned returns.
    ## mean returns over each return time series 
    mean_returns = np.mean(returns.values, axis=0)
    
    ## X: serially demeaned returns.
    X = ret_matrix - mean_returns
    
    # 3. Diagonal Sample Covariance matrix elements.
    # variance = Cii                                        ----> array([0.01943418, 0.01834617, 0.03849106])
    variance = (1/(returns.shape[0]-1))*np.sum(X*X, axis=0)
    std = np.sqrt(variance)
    
    # 4. Calculate the normalized demeaned returns.
    Y = X / std
    
    # 5.Keep only the first M values in Y.                   ----> (501, 3)
    Y = Y[:-1, :]
    
    # 6.Cross-sectionally demean  %%%%% COMPLETAMENTE SEGURO HASTA ACÁ %%%%%
    if rm_overall:
        Y = (Y.T - np.mean(Y, axis=1)).T
        Y = Y[:-1, :]
        
    # 8. Take the alpha expected returns Ei and normalize them.
    exp_ret = returns.apply(lambda x: x.rolling(50).mean()).values[-1]
    exp_ret_nor = np.matrix(exp_ret / std).T
    
    # 9.Calculate the residuals epsilon_i
    w = Y @ exp_ret_nor
    w = np.linalg.inv(Y @ Y.T) @ w
    w = exp_ret_nor - Y.T @ w
    w = w.flatten() / std
    w = w / np.sum(np.abs(w))
    w = pd.DataFrame(w.flatten(), columns=returns.columns, index=["Weight"])
    
    return w.T

## Traer la  data de las acciones

In [3]:
# paths to read and storage data
folderPath_rsrc = '/Resources/'
folderPath_results = '/Results/'
cwd = os.getcwd()
path_rsrc = cwd + folderPath_rsrc
path_results = cwd + folderPath_results

In [4]:
tickers = pd.read_excel(path_rsrc + "rusell2000.xlsx", header=3)["Ticker"].to_list()
tickers = tickers[:20]
tickers

['TGNA',
 'PECO',
 'ADNT',
 'FCFS',
 'ALIT',
 'SHLS',
 'CADE',
 'AVNT',
 'MTSI',
 'EPRT',
 'DUOL',
 'CNMD',
 'STAA',
 'LTHM',
 'NARI',
 'APLE',
 'TWNK',
 'AEIS',
 'STNE',
 'SMPL']

In [5]:
close_df = yf.download(tickers, start="2021-01-01", end="2023-01-01")["Adj Close"]
close_df[100:].head()

[*********************100%***********************]  20 of 20 completed


,ADNT,AEIS,ALIT,APLE,AVNT,CADE,CNMD,DUOL,EPRT,FCFS,LTHM,MTSI,NARI,PECO,SHLS,SMPL,STAA,STNE,TGNA,TWNK
Date,,,,,,,,,,,,,,,,,,,,
2021-05-27,49.869999,101.084991,10.43,14.942309,49.775753,27.242062,134.555771,NaN,23.279364,78.044792,19.879999,59.040001,88.820000,6.295607,27.180000,34.439999,143.300003,66.320000,18.746752,15.550000
2021-05-28,50.060001,101.094902,10.40,14.811644,49.794910,27.358368,135.699005,NaN,23.481161,77.375092,19.510000,59.200001,86.949997,6.295607,27.600000,34.529999,146.029999,65.970001,18.660139,15.680000
2021-06-01,52.500000,102.571533,10.38,15.259633,50.685818,27.599924,136.448029,NaN,24.196600,78.733917,19.889999,58.279999,81.449997,6.295607,27.950001,34.459999,143.949997,65.139999,19.093203,16.049999
2021-06-02,50.619999,102.541809,10.37,15.175634,49.651218,27.501513,131.254196,NaN,24.325012,79.199783,20.309999,58.169998,80.760002,6.295607,27.920000,34.540001,143.669998,66.070000,18.554283,16.469999
2021-06-03,52.930000,99.905655,10.31,14.923643,49.756592,27.707283,131.402054,NaN,24.820318,79.248329,19.969999,57.650002,79.620003,6.821808,25.780001,33.980000,140.119995,63.840000,18.317335,16.389999


### Calcular los retornos

In [6]:
returns = close_df.pct_change()
returns.dropna(inplace=True)

In [7]:
returns.head()

,ADNT,AEIS,ALIT,APLE,AVNT,CADE,CNMD,DUOL,EPRT,FCFS,LTHM,MTSI,NARI,PECO,SHLS,SMPL,STAA,STNE,TGNA,TWNK
Date,,,,,,,,,,,,,,,,,,,,
2021-07-29,0.052842,0.006690,-0.008791,0.012718,0.016270,0.011245,-0.007691,0.001378,0.028315,0.014807,0.021488,0.005506,0.002676,0.006133,-0.003842,0.001072,0.002667,0.033456,0.004535,0.009926
2021-07-30,0.002141,0.013976,0.043237,-0.011897,-0.004105,-0.010736,-0.000797,0.043216,0.000672,-0.003774,0.001026,0.024228,-0.001557,0.003944,0.019986,0.003481,0.000626,-0.002712,0.000000,-0.011671
2021-08-02,-0.018989,-0.007229,0.026567,-0.012040,-0.022259,-0.003488,-0.013992,0.025312,-0.010403,0.005934,0.027678,-0.003240,-0.006460,0.002500,-0.014782,-0.013874,0.002580,-0.004419,-0.009594,-0.008701
2021-08-03,-0.001210,-0.001456,0.005176,-0.008802,0.012437,0.017114,-0.015734,-0.027399,0.001695,0.014560,-0.007980,-0.009753,-0.002354,-0.004275,0.064550,-0.002164,-0.003119,-0.025947,-0.002849,-0.010031
2021-08-04,-0.035853,-0.121342,0.014418,-0.025956,-0.038934,-0.014532,-0.043400,0.011726,-0.005755,-0.002103,-0.036199,0.007879,0.037303,0.001789,0.004589,-0.025759,0.007040,-0.007361,-0.012000,-0.039265


### Aplicar estrategia

In [8]:
estrategia = weights(returns)

In [9]:
estrategia

,Weight
ADNT,0.006687
AEIS,-0.011993
ALIT,0.038138
APLE,-0.077258
AVNT,0.010035
CADE,-0.052592
CNMD,0.037756
DUOL,0.081917
EPRT,-0.041868
FCFS,-0.070146
